# HOMEWORK: k-Nearest Neighbors

In [1]:
!pip install -U scikit-learn


  Using cached scikit_learn-0.18.1-cp27-cp27m-win_amd64.whl
  Found existing installation: scikit-learn 0.17.1
    Uninstalling scikit-learn-0.17.1:
      Successfully uninstalled scikit-learn-0.17.1


    DEPRECATION: Uninstalling a distutils installed project (scikit-learn) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import os
import scipy
import numpy
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 100)

from sklearn import preprocessing, neighbors, grid_search
from sklearn import cross_validation

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
df = pd.read_csv('dataset-boston.csv')

In [4]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


The Boston dataset concerns itself with housing values in suburbs of Boston.  A description of the dataset is as follows:

- CRIM: per capita crime rate by town
- ZN: proportion of residential land zoned for lots over 25,000 sqft
- INDUS: proportion of non-retail business acres per town
- CHAS: Charles River binary/dummy variable (= 1 if tract bounds river; 0 otherwise)
- NOX: nitric oxides concentration (parts per 10 million)
- RM: average number of rooms per dwelling
- AGE: proportion of owner-occupied units built prior to 1940
- DIS: weighted distances to five Boston employment centers
- RAD: index of accessibility to radial highways
- TAX: full-value property-tax rate (per ten thousands of dollars)
- PTRATIO: pupil-teacher ratio by town
- B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
- LSTAT: % lower status of the population
- MEDV: Median value of owner-occupied homes (in thousands of dollars)

## Question 1.  
+ Let's first categorize `MEDV` to 4 groups: Bottom 20% as Level 1, next 30% as Level 2, next 30% categorized as Level 3, and the top 20% as Level 4.  
+ Please create a new variable `MEDV_Category` that stores the level number
+ Remember the quantile function
+ Remember how to segment your pandas data frame

In [5]:
df['MEDV_Category'] = ""
val = ""
q20 = df['MEDV'].quantile(q=0.20)
q50 = df['MEDV'].quantile(q=0.50)
q80 = df['MEDV'].quantile(q=0.80)
for i,row in df.iterrows():
    if row['MEDV'] < q20:
        val = "Level 1"
        df.set_value(i,'MEDV_Category',val)
    if row['MEDV'] < q50 and row['MEDV'] >= q20:
        val = "Level 2"
        df.set_value(i,'MEDV_Category',val)
    if row['MEDV'] < q80 and row['MEDV'] >= q50:
        val = "Level 3"
        df.set_value(i,'MEDV_Category',val)
    if row['MEDV'] > q80:
        val = "Level 4"
        df.set_value(i,'MEDV_Category',val)
    else: continue
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV,MEDV_Category
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,Level 3
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,Level 3
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,Level 4
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,Level 4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,Level 4


### Our goal is to predict `MEDV_Category` based on `RM`, `PTRATIO`, and `LSTAT`

## Question 2.  

+ First normalize `RM`, `PTRATIO`, and `LSTAT`.  
+ By normalizing, we mean to scale each variable between 0 and 1 with the lowest value as 0 and the highest value as 1

+ Check out the documentation for MinMaxScaler()

In [6]:
df['RM_N'] = 0.0
df['PTRATIO_N'] = 0.0
df['LSTAT_N'] = 0.0
df['RM_N'] = preprocessing.MinMaxScaler(feature_range=(0,1)).fit_transform(df['RM'])
df['PTRATIO_N'] = preprocessing.MinMaxScaler().fit_transform(df['PTRATIO'])
df['LSTAT_N'] = preprocessing.MinMaxScaler().fit_transform(df['LSTAT'])
df.head()

C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Resha

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,BLACK,LSTAT,MEDV,MEDV_Category,RM_N,PTRATIO_N,LSTAT_N
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,Level 3,0.577505,0.287234,0.089680
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,Level 3,0.547998,0.553191,0.204470
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,Level 4,0.694386,0.553191,0.063466
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,Level 4,0.658555,0.648936,0.033389
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2,Level 4,0.687105,0.648936,0.099338


In [7]:
df2 = df[['MEDV_Category', 'RM_N', 'PTRATIO_N', 'LSTAT_N']].copy()
df2.head()

,MEDV_Category,RM_N,PTRATIO_N,LSTAT_N
0,Level 3,0.577505,0.287234,0.089680
1,Level 3,0.547998,0.553191,0.204470
2,Level 4,0.694386,0.553191,0.063466
3,Level 4,0.658555,0.648936,0.033389
4,Level 4,0.687105,0.648936,0.099338


## Question 3.  

+ Run a k-NN classifier with 5 nearest neighbors and report your misclassification error; set weights to uniform
+ Calculate your misclassification error on the training set

In [8]:
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform')
knn.fit(df2.iloc[:,1:], df2.MEDV_Category)

true = knn.score(df2.iloc[:,1:], df2.MEDV_Category)
mis = 1 - true
print 'Misclassification rate is', mis

Misclassification rate is 0.211462450593


Answer: TODO

## Question 4. 
+ Is this error reliable? 
+ What could we do to make it better?

yes, since we have normalized the data, the error is reliable.
We can change number of neighbors to higher or lower to see if the misclassification rate goes up or down. 

## Question 5.  
+ Now use 10-fold cross-validation to choose the most efficient `k`

In [9]:
from sklearn import grid_search, cross_validation
import matplotlib.pyplot as plt

k = range(1, 100)# some *list of numbers*! Make this go to 100!
params = {"n_neighbors": k}# Specify the parameter you're changing as a dictionary key
            # Specify the *list of numbers* as the dictionary value!

kf = cross_validation.KFold(len(df2), n_folds = 10)
gs = grid_search.GridSearchCV(
    estimator=neighbors.KNeighborsClassifier(weights='uniform'),
    param_grid=params,
    cv=kf,
)
gs.fit(df2.iloc[:,1:], df2.MEDV_Category)
gs.grid_scores_

[mean: 0.60474, std: 0.09211, params: {'n_neighbors': 1},
 mean: 0.64229, std: 0.08556, params: {'n_neighbors': 2},
 mean: 0.65020, std: 0.09816, params: {'n_neighbors': 3},
 mean: 0.65810, std: 0.08049, params: {'n_neighbors': 4},
 mean: 0.65217, std: 0.09627, params: {'n_neighbors': 5},
 mean: 0.66601, std: 0.08487, params: {'n_neighbors': 6},
 mean: 0.66996, std: 0.09256, params: {'n_neighbors': 7},
 mean: 0.66601, std: 0.09741, params: {'n_neighbors': 8},
 mean: 0.66798, std: 0.09654, params: {'n_neighbors': 9},
 mean: 0.66008, std: 0.10081, params: {'n_neighbors': 10},
 mean: 0.65613, std: 0.09384, params: {'n_neighbors': 11},
 mean: 0.65217, std: 0.08690, params: {'n_neighbors': 12},
 mean: 0.65613, std: 0.09673, params: {'n_neighbors': 13},
 mean: 0.65415, std: 0.09640, params: {'n_neighbors': 14},
 mean: 0.66798, std: 0.09715, params: {'n_neighbors': 15},
 mean: 0.66601, std: 0.10372, params: {'n_neighbors': 16},
 mean: 0.66403, std: 0.09415, params: {'n_neighbors': 17},
 mean:

## Question 6.  

+ Explain your findings
+ What were your best parameters?
+ What was the best k?
+ What was the best model?

The mean is the accuracy of the K-nearest neighbor classifier, for each value of n_neighbors
 mean: 0.68577, std: 0.08879, params: {'n_neighbors': 22}
k = 22 - seems a bit high though
What was the best model? I didn't quite understand this question, we are using only the kNN classifier model here

## Question 7.  

+ Train your model with the optimal `k` you found above 
+ (don't worry if it changes from time to time - if that is the case use the one that is usually the best)

In [10]:
knn2 = neighbors.KNeighborsClassifier(n_neighbors=22, weights='uniform')
knn2.fit(df2.iloc[:,1:], df2.MEDV_Category)

true2 = knn2.score(df2.iloc[:,1:], df2.MEDV_Category)
mis2 = 1 - true2
print 'Misclassification rate is', mis2

Misclassification rate is 0.264822134387


Answer: TODO

## Question 8.  

+ After training your model with that `k`, 
+ use it to *predict* the class of a neighborhood with `RM = 2`, `PRATIO = 19`, and `LSTAT = 3.5`
+ If you are confused, check out the sklearn documentation for KNN

In [11]:
#first, i will try to normalize these values
RM_max = df['RM'].max()
RM_min = df['RM'].min()
PTRATIO_max = df['PTRATIO'].max()
PTRATIO_min = df['PTRATIO'].min()
LSTAT_max = df['LSTAT'].max()
LSTAT_min = df['LSTAT'].min()

RM_predict = (2 - RM_min)/(RM_max - RM_min)
PTRATIO_predict = (19 - PTRATIO_min)/(PTRATIO_max - PTRATIO_min)
LSTAT_predict = (3.5 - LSTAT_min)/(LSTAT_max - LSTAT_min)

In [12]:
pred = [RM_predict, PTRATIO_predict, LSTAT_predict]
pred

#now without normalization
no_norm = [2,19,3.5]

In [17]:
print "The predicted MEDV_Category is ", knn2.predict(pred)
#getting Level 2 as the answer, but it is interesting that the normalization value of RM is less than 0.
#Level 2 is predict for input with or without normalization

The predicted MEDV_Category is  ['Level 2']


C:\Users\niyer\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


Answer: TODO